### Data loading and cleaning (if necessary)



Task 0a: Data cleaning

In [6]:
# Load all libraries
import os
from requests import get
from urllib.parse import urlparse
import numpy as np
import pandas as pd
import geopandas as gpd

In [7]:
# Function to download files from remote
def cache_data(src:str, dest:str) -> str:
    """
    Create a folder to store file from URL.
    If folder doesn't already exists, then create one, before writing the file.

    src : URL
    dest : location on local drive
    
    """    
    url = urlparse(src) # We assume that this is some kind of valid URL 
    fn  = os.path.split(url.path)[-1] # Extract the filename
    dfn = os.path.join(dest,fn) # Destination filename as path
    
    if not os.path.isfile(dfn):
        
        print(f"{dfn} not found, downloading!")

        path = os.path.split(dest)
        
        if len(path) >= 1 and path[0] != '':
            os.makedirs(os.path.join(*path), exist_ok=True)
            
        with open(dfn, "wb") as file:
            response = get(src)
            file.write(response.content)
            
        print("\tDone downloading...")

    else:
        print(f"Found {dfn} locally!")

    return dfn

Task 0b: Load in CSV files

In [ ]:
# Read in airbnb, crime, and lsoa stats files. https://www.dropbox.com/home/casa/casa_fsds_cheers

listings =
crime = 
lsoa_stat = 

Task 0c: Load in London spatial data (boroughs and LSOA)

In [28]:
ddir  = os.path.join('data','geo') # destination directory
spath = 'https://github.com/jreades/i2p/blob/master/data/src/' # source path
lsoa_url = r'https://www.dropbox.com/scl/fi/hda7eykjtjp760s1u4z05/LSOA_Dec_2021_Boundaries_Generalised_Clipped_EW_BGC_2022_5605507071095448309.geojson'

# boros = gpd.read_file( cache_data(spath+'Boroughs.gpkg?raw=true', ddir) )
water = gpd.read_file( cache_data(spath+'Water.gpkg?raw=true', ddir) )
green = gpd.read_file( cache_data(spath+'Greenspace.gpkg?raw=true', ddir) )
lsoa = gpd.read_file( cache_data(lsoa_url, ddir) )

print('Done.')

Found data/geo/Water.gpkg locally!
Found data/geo/Greenspace.gpkg locally!


DriverError: 'data/geo/LSOA_Dec_2021_Boundaries_Generalised_Clipped_EW_BGC_2022_5605507071095448309.geojson' not recognized as a supported file format.

In [ ]:
# Reproject everything to British National Grid


In [ ]:
# Plot London map (empty)
fig, ax = plt.subplots(1,1, figsize=(12,9))

# Plot all three GeoPackages to the same axes
lsoa.plot(edgecolor='red', facecolor='white',ax=ax)
green.plot(edgecolor=None, facecolor=(0,1,0,0.5), ax=ax)
water.plot(edgecolor=None,facecolor='#a2cffe',ax=ax)

# Set the x and y limits
ax.set_xlim([501000, 563000])
ax.set_ylim([155000, 202000])

### Descriptive statistics (Question #6 - 18 points)

*Objective:* 
Show the clustering of airbnb (by type), and crime incidents (by type). Show possible correlations between airbnb density and crime incidents
Approach: 

1. Split airbnb listing by 3 types (entire house/private room/shared room), and crime by 2 types (person vs property)
2. Plot out all the airbnb listings.
     - Heat map (KDE for each)
     - Descriptive statistics for each (LSOA min, max, mean, median)
3. Plot all the crime incidents, split by .
    - Heat map (KDE for each)
    - Descriptive statistics for each (LSOA min, max, mean, median)
4. Visual observation based on the heatmaps above where airbnb listings and crimes seem to overlap
5. Run Pearson correlations: (See QM practical 3-4)

*Notes:* 

- [From Kernel Density Estimation to Spatial Analysis In Python](https://towardsdatascience.com/from-kernel-density-estimation-to-spatial-analysis-in-python-64ddcdb6bc9b)
-------------------------------------------------------------------------------

Task 1: Split airbnb listing by 3 types (entire house/private room/shared room), and crime by 2 types (person vs property)


Task 2: Plot out all the airbnb listings. Produce heat map for each type

Task 3: Plot out all the crime incidents. Produce heat map for each type

Task 4: Visual observation based on the heatmaps above where airbnb listings and crimes seem to overlap

Task 5: Run Pearson correlations: (See QM practical 3-4)

### Inferring statistics (Question #7 - 45 points)

*Objective:* 
Show causation and infer recommendations

1. Step 1
2. Step 2
3. Step 3

*Notes:* 

- [From Kernel Density Estimation to Spatial Analysis In Python](https://towardsdatascience.com/from-kernel-density-estimation-to-spatial-analysis-in-python-64ddcdb6bc9b)
- [Airbnb and neighborhood crime: The incursion of tourists or the erosion of local social dynamics?](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8279333/): "We find evidence that increases in Airbnb listings led to more violence in neighborhoods in later years. This result supports the notion that the prevalence of Airbnb listings erodes the natural ability of a neighborhood to prevent crime, but does not support the interpretation that elevated numbers of tourists bring crime with them."
-------------------------------------------------------------------------------